<a href="https://colab.research.google.com/github/haiiro1/Capstone/blob/feature%2Fjuako/Preproceso_de_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp "/content/drive/My Drive/cv2_cuda/cv2.cpython-312-x86_64-linux-gnu.so" .

In [ ]:
import cv2
cv2.__version__

'4.13.0-dev'

In [ ]:
import os
import cv2
import glob
import numpy as np
from tqdm import tqdm


class ImagePreprocessorCuda:
    def __init__(self, high_contrast=1.5, low_contrast=0.7):
        self.high_contrast = high_contrast
        self.low_contrast = low_contrast

    def process_image(self, img_path):
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Skipping (could not read): {img_path}")
            return

        gpu_img = cv2.cuda_GpuMat()
        gpu_img.upload(img)

        scalar_high_contrast = cv2.cuda_GpuMat(gpu_img.size(), gpu_img.type())
        scalar_high_contrast.setTo(self.high_contrast)
        gpu_high = cv2.cuda.multiply(gpu_img, scalar_high_contrast)

        scalar_low_contrast = cv2.cuda_GpuMat(gpu_img.size(), gpu_img.type())
        scalar_low_contrast.setTo(self.low_contrast)
        gpu_low = cv2.cuda.multiply(gpu_img, scalar_low_contrast)

        high = gpu_high.download()
        low = gpu_low.download()

        dirname, filename = os.path.split(img_path)
        name, ext = os.path.splitext(filename)

        cv2.imwrite(os.path.join(dirname, f"{name}_high{ext}"), high)
        cv2.imwrite(os.path.join(dirname, f"{name}_low{ext}"), low)

        angles = {"rot90": 90, "rot180": 180, "rot270": 270}
        h, w = img.shape[:2]
        center = (w / 2, h / 2)

        for suffix, angle in angles.items():
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            gpu_rot = cv2.cuda.warpAffine(gpu_img, M, (w, h))
            rotated = gpu_rot.download()
            cv2.imwrite(os.path.join(dirname, f"{name}_{suffix}{ext}"), rotated)

    def process_directories(self, root_dir, extensions=(".jpg", ".png", ".jpeg")):
        for ext in extensions:
            img_paths = glob.glob(os.path.join(root_dir, "**", f"*{ext}"), recursive=True)
            for img_path in tqdm(img_paths, desc=f"Processing {ext} images in {root_dir}"):
                self.process_image(img_path)


def setup_kaggle_dataset(dataset_name, dataset_dir="/content/dataset"):

    os.makedirs(dataset_dir, exist_ok=True)
    os.system(f"kaggle datasets download -d {dataset_name} -p {dataset_dir} --unzip")
    return dataset_dir


def main():
    kaggle_dataset = "jegodoy/tomato-dataset-for-capstone"
    root_dir = setup_kaggle_dataset(kaggle_dataset)

    processor = ImagePreprocessorCuda()

    print(f"✅ Dataset ready at: {root_dir}")

    sub_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

    if not sub_dirs:
        print("⚠️ No subdirectories found. Processing root directory only.")
        sub_dirs = [root_dir]

    for img_dir in sub_dirs:
        print(f"🚀 Processing directory: {img_dir}")
        processor.process_directories(img_dir)

    print("🎉 Preprocessing complete!")


if __name__ == "__main__":
    main()

✅ Dataset ready at: /content/dataset
🚀 Processing directory: /content/dataset/tomato


Processing .jpg images in /content/dataset/tomato: 100%|██████████| 205314/205314 [17:11<00:00, 198.96it/s]
Processing .png images in /content/dataset/tomato: 0it [00:00, ?it/s]
Processing .jpeg images in /content/dataset/tomato: 0it [00:00, ?it/s]

🎉 Preprocessing complete!


In [ ]:
import shutil
import os
from google.colab import files

folder_path = '/content/dataset/tomato'

zip_path = '/content/tomato.zip'
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)


files.download(zip_path)